# Problem Statement
Write a function that takes as input two timestamps of the form 2017/05/13 12:00 and calculates their differences in hours.
Please only return the full hour difference and round the results.
E.g., 2022/02/15 00:05 and 2022/02/15 01:00 would return 1 hour.
 
Once you did this, please expand the above function so that only the time difference will be counted between 09:00 – 17:00 and only on weekdays.
 
Let the user choose what function to use.

In [3]:
# setup
import pendulum as pdl

WEEKEND = ["Saturday", "Sunday"]
WEEKDAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
# expected input format
EXPECTED_FORMAT = 'YYYY/MM/DD HH:mm'
# end of day and start of day hours and minutes
EOD_HOUR = 17
EOD_MINUTES = 0
SOD_HOUR = 9
SOD_MINTUES = 0
WORKING_TIME_PER_DAY = EOD_HOUR - SOD_HOUR

def remaining_weekdays(end_timestamp: pdl.datetime, remaining_days: int) -> int:
    """
    :param end_timestamp: end date
    :param remaining_days: number of days that are less than a week
    :return: number of weekdays in these days
    """
    # do not calculate last date, current index should start from date before it
    current_index = WEEKDAYS.index(end_timestamp.format('dddd')) - 1
    result = 0
    # loop through remaining days backwards
    for i in range(remaining_days):
        current_day = WEEKDAYS[current_index]
        if is_weekday(current_day, False):
            result += 1
        current_index -= 1
    return result

def is_weekday(day: str, formatting: bool = True) -> bool:
    """
    :param day: current day, or current timestamp
    :param formatting: true means format timestamp
    :return: true if weekday
    """
    if formatting:
        day = day.format('dddd')
    return day not in WEEKEND

def convert_to_range(timestamp: pdl.datetime) -> pdl.datetime:
    """
    :param timestamp: input datetime
    :return: timestamp transformed to working time range
    """
    if timestamp.hour > EOD_HOUR:
        timestamp = timestamp.set(hour=EOD_HOUR, minute=EOD_MINUTES)
    elif timestamp.hour < SOD_HOUR:
        timestamp = timestamp.set(hour=SOD_HOUR, minute=SOD_MINTUES)
    return timestamp

def timestamps_diff_rounded(timestamp1_string: str, timestamp2_string: str, only_workdays: bool = False) -> int:
    """
    :param timestamp1_string: input datetime
    :param timestamp2_string: input datetime
    :param only_workdays: true means the time difference will be counted between 09:00 – 17:00 and only on weekdays.
    :return: int difference in hours
    """
    # convert the input strings into datetime from expected format
    timestamp1 = pdl.from_format(timestamp1_string, EXPECTED_FORMAT)
    timestamp2 = pdl.from_format(timestamp2_string, EXPECTED_FORMAT)
    start_timestamp = min(timestamp1, timestamp2)
    end_timestamp = max(timestamp1, timestamp2)
    hours = 0

    # first case: normal calculation
    if not only_workdays:
        return round(start_timestamp.diff(end_timestamp).in_minutes() / 60)

    # second case: workdays, during working hours
    # adjust the time to become in range of working hours
    start_timestamp = convert_to_range(start_timestamp)
    end_timestamp = convert_to_range(end_timestamp)
    
    # if same date subtract directly
    if(is_weekday(start_timestamp) and start_timestamp.year == end_timestamp.year and start_timestamp.month == end_timestamp.month and  start_timestamp.day == end_timestamp.day):
        return round(start_timestamp.diff(end_timestamp).in_minutes() / 60)
        
    # diff() by default returns the absolute difference, then we convert it to days
    # remove first and last date from calculations
    total_days = start_timestamp.diff(end_timestamp).in_days() - 1
    # remaining days < 7
    weekdays = (total_days // 7) * 5
    hours = weekdays * WORKING_TIME_PER_DAY

    # account for first date
    if is_weekday(start_timestamp):
        # worked hours on start date = subtract start date from EOD timestamp
        hours += round(pdl.datetime(year=start_timestamp.year, month=start_timestamp.month, day=start_timestamp.day, hour=EOD_HOUR, minute=EOD_MINUTES).diff(start_timestamp).in_minutes() / 60)

    # calculate hours in remaining days
    remaining_days = total_days % 7
    hours += remaining_weekdays(end_timestamp, remaining_days) * WORKING_TIME_PER_DAY

    # account for last date
    if is_weekday(end_timestamp):
        # worked hours on end date = subtract end date from SOD timestamp
        hours += round(pdl.datetime(year=end_timestamp.year, month=end_timestamp.month, day=end_timestamp.day, hour=SOD_HOUR, minute=SOD_MINTUES).diff(end_timestamp).in_minutes() / 60)
    return hours

In [4]:
timestamp1_string = "2017/05/4 10:05"
timestamp2_string = "2017/05/6 15:00"
result = timestamps_diff_rounded(timestamp1_string, timestamp2_string)
print("Result:", result, "hours")
result = timestamps_diff_rounded(timestamp1_string, timestamp2_string, True)
print("Result:", result, "hours")

Result: 53 hours
Result: 15 hours
